# CSE547 - Colab 2
## Frequent Pattern Mining in Spark

Adapted from Stanford CS246

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 14.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=c1e388df9cf87b56ea50bdad1528862f79975cabec5b0a584a5bb20ffcfaf46f
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 23 not

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

Let's initialize the Spark context.

In [ ]:
#sc.stop()
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [ ]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2023-01-19 22:31:47--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  59.6MB/s    in 0.2s    

2023-01-19 22:31:47 (59.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [ ]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [ ]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [ ]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer). 

In [ ]:

group_order=orders.join(products, on='product_id').groupBy("order_id").agg(collect_list('product_name').alias("product_name"))


In [ ]:
group_order.count()

131209

In [ ]:
group_order.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_name: array (nullable = false)
 |    |-- element: string (containsNull = false)



In [ ]:
group_order.limit(5).collect()

[Row(order_id=1, product_name=['Bulgarian Yogurt', 'Organic 4% Milk Fat Whole Milk Cottage Cheese', 'Organic Celery Hearts', 'Cucumber Kirby', 'Lightly Smoked Sardines in Olive Oil', 'Bag of Organic Bananas', 'Organic Hass Avocado', 'Organic Whole String Cheese']),
 Row(order_id=96, product_name=['Roasted Turkey', 'Organic Cucumber', 'Organic Grape Tomatoes', 'Organic Pomegranate Kernels', 'Organic Raspberries', 'Organic Whole Strawberries', 'Organic Blueberries']),
 Row(order_id=112, product_name=['Fresh Cauliflower', 'I Heart Baby Kale', 'Sea Salt Baked Potato Chips', 'Marinara Pasta Sauce', 'Organic Hass Avocado', 'Organic Lemon', 'Coconut Water Kefir', 'Premium Epsom Salt', 'Umcka Elderberry Intensive Cold + Flu Berry Flavor', 'Hickory Honey Barbeque Baked Potato Chips', 'Baked Sea Salt & Vinegar Potato Chips']),
 Row(order_id=218, product_name=['Natural Artisan Water', 'Okra', 'Organic Yellow Peaches', 'Black Plum', 'Citrus Mandarins Organic']),
 Row(order_id=456, product_name=['C

In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with 

```minSupport=0.01``` and ```minConfidence=0.5```



In [ ]:
# YOUR CODE HERE

from pyspark.ml.fpm import FPGrowth
fpgrowth=FPGrowth(itemsCol="product_name",minSupport=0.01 , minConfidence=0.5)
model= fpgrowth.fit(group_order)
# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(group_order).show()

+--------------------+-----+
|               items| freq|
+--------------------+-----+
|      [Green Onions]| 1445|
|   [Red Raspberries]| 1493|
|    [Organic Banana]| 2332|
|  [Jalapeno Peppers]| 1899|
|[Organic Large Ex...| 2891|
|[Organic Whole St...| 1993|
|[Organic Peeled W...| 2460|
|             [Limes]| 6033|
|[Limes, Large Lemon]| 1595|
|     [Limes, Banana]| 1331|
|       [Raspberries]| 3279|
|      [Hass Avocado]| 1633|
|[Organic Broccoli...| 1361|
|[Uncured Genoa Sa...| 1788|
|      [Spring Water]| 2225|
|[Michigan Organic...| 2627|
|     [Yellow Onions]| 3762|
|[Organic Strawber...|10894|
|[Organic Strawber...| 3074|
|[Organic Strawber...| 2174|
+--------------------+-----+
only showing top 20 rows

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+

+--------+--------------------+----------+
|order_id|        product_name|prediction|

In [ ]:
model.associationRules.show()


+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



In [ ]:
# YOUR CODE HERE
model.freqItemsets.count()
model.associationRules.count()

0

Compute how many Frequent Itemsets and Association Rules were generated by running FP-growth.


In [ ]:
# YOUR CODE HERE

from pyspark.ml.fpm import FPGrowth
fpgrowth=FPGrowth(itemsCol="product_name",minSupport=0.001 , minConfidence=0.5)
model= fpgrowth.fit(group_order)
# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(group_order).show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|[White Cheddar Po...| 370|
|[Organic YoKids V...| 259|
|[Total 0% Nonfat ...| 993|
|[Total 0% Nonfat ...| 137|
|[Total 0% Nonfat ...| 157|
|[Total 0% Nonfat ...| 143|
|[Total 0% Nonfat ...| 258|
|[Organic Large Gr...| 730|
|[Organic Large Gr...| 185|
|[Organic Large Gr...| 263|
|[Organic Large Gr...| 136|
|[Organic Large Gr...| 164|
|[Organic Cream Ch...| 365|
|[Organic Reduced ...| 140|
|[Florida Orange J...| 283|
|      [Green Onions]|1445|
|[Green Onions, Li...| 206|
|[Green Onions, Or...| 133|
|[Green Onions, Bu...| 187|
|[Green Onions, La...| 229|
+--------------------+----+
only showing top 20 rows

+--------------------+--------------------+------------------+------------------+--------------------+
|          antecedent|          consequent|        confidence|              lift|             support|
+--------------------+--------------------+------------------+------------------+---------------

Now retrain the FP-growth model changing only 
```minsupport=0.001``` 
and compute how many Frequent Itemsets and Association Rules were generated.

In [ ]:
# YOUR CODE HERE
model.freqItemsets.count()
model.associationRules.count()

11

In [ ]:
model.freqItemsets.sort(model.freqItemsets.freq.desc()).show(truncate=False)

+------------------------+-----+
|items                   |freq |
+------------------------+-----+
|[Banana]                |18726|
|[Bag of Organic Bananas]|15480|
|[Organic Strawberries]  |10894|
|[Organic Baby Spinach]  |9784 |
|[Large Lemon]           |8135 |
|[Organic Avocado]       |7409 |
|[Organic Hass Avocado]  |7293 |
|[Strawberries]          |6494 |
|[Limes]                 |6033 |
|[Organic Raspberries]   |5546 |
|[Organic Blueberries]   |4966 |
|[Organic Whole Milk]    |4908 |
|[Organic Cucumber]      |4613 |
|[Organic Zucchini]      |4589 |
|[Organic Yellow Onion]  |4290 |
|[Organic Garlic]        |4158 |
|[Seedless Red Grapes]   |4059 |
|[Asparagus]             |3868 |
|[Organic Grape Tomatoes]|3823 |
|[Organic Red Onion]     |3818 |
+------------------------+-----+
only showing top 20 rows



In [ ]:
model.transform(group_order).sort(model.transform.prediction.desc()).show()

AttributeError: ignored

In [ ]:
model.associationRules.show(truncate=False)

+-----------------------------------------------------------------+------------------------+------------------+------------------+---------------------+
|antecedent                                                       |consequent              |confidence        |lift              |support              |
+-----------------------------------------------------------------+------------------------+------------------+------------------+---------------------+
|[Organic Whole String Cheese, Organic Hass Avocado]              |[Bag of Organic Bananas]|0.5314685314685315|4.504745125675359 |0.0011584571180330617|
|[Organic Broccoli, Organic Hass Avocado]                         |[Bag of Organic Bananas]|0.5048231511254019|4.278897986822536 |0.001196564260073623 |
|[Organic Navel Orange, Organic Raspberries]                      |[Bag of Organic Bananas]|0.5412186379928315|4.587387356098284 |0.0011508356896249496|
|[Organic Kiwi, Organic Hass Avocado]                             |[Bag of Organic

Now, for 
```minsupport=0.001```, 
visualize the top Frequent Itemsets and Association Rules.

In addition, for
```minsupport=0.001```, 
calculate the "interest" of the Association Rules. 

(Hint: One way to do this is by joining the Frequent Itemsets with the Association Rules, then introducing an "interest" column using the joined dataframe)




In [ ]:
# YOUR CODE HERE
result = model.freqItemsets.join(model.associationRules, model.freqItemsets.items == model.associationRules.antecedent)

result.show(truncate=False)
#result.select(freq)
# number of unique orders

+-----------------------------------------------------------------+----+-----------------------------------------------------------------+------------------------+------------------+------------------+---------------------+
|items                                                            |freq|antecedent                                                       |consequent              |confidence        |lift              |support              |
+-----------------------------------------------------------------+----+-----------------------------------------------------------------+------------------------+------------------+------------------+---------------------+
|[Organic Broccoli, Organic Hass Avocado]                         |311 |[Organic Broccoli, Organic Hass Avocado]                         |[Bag of Organic Bananas]|0.5048231511254019|4.278897986822536 |0.001196564260073623 |
|[Organic D'Anjou Pears, Organic Hass Avocado]                    |352 |[Organic D'Anjou Pears, Organic 

In [ ]:
311/131209

0.002370264234922909

In [ ]:
result.show(truncate=False)

+-----------------------------------------------------------------+----+-----------------------------------------------------------------+------------------------+------------------+------------------+---------------------+
|items                                                            |freq|antecedent                                                       |consequent              |confidence        |lift              |support              |
+-----------------------------------------------------------------+----+-----------------------------------------------------------------+------------------------+------------------+------------------+---------------------+
|[Organic Broccoli, Organic Hass Avocado]                         |311 |[Organic Broccoli, Organic Hass Avocado]                         |[Bag of Organic Bananas]|0.5048231511254019|4.278897986822536 |0.001196564260073623 |
|[Organic D'Anjou Pears, Organic Hass Avocado]                    |352 |[Organic D'Anjou Pears, Organic 

To conclude, go to Gradescope and read the remaining questions. We will ask you to inspect the resulting dataframes, and report a few results.